In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import geopandas as gpd
import pandas as pd
import os
import urllib
import zipfile
import re
from descartes import PolygonPatch
from pysal.model import mgwr
import pysal.lib
import libpysal as ps
from libpysal.weights.contiguity import Queen
from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
# conda install mgwr
from mgwr.gwr import GWR, MGWR
from mgwr.sel_bw import Sel_BW
from mgwr.utils import shift_colormap, truncate_colormap
import warnings
#warnings.filterwarnings("ignore")

In [2]:
from pysal.model import spreg

In [3]:
from spreg import ML_Error
from spreg import ML_Lag

In [4]:
import libpysal as lps
from libpysal.weights import Queen, Rook, KNN
from libpysal import weights
import mapclassify

In [5]:
dfh = pd.read_csv("Data_excel/health.csv")
dfu = pd.read_csv("Data_excel/unemployment_rate_3.csv")

In [6]:
dfh[~dfh.isin(dfu)].dropna(how = 'all', inplace= True)

In [7]:
dfhg=dfh.groupby("Local authority name (2020)").mean()

D:\conda_tmp\ipykernel_6124\3341374521.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dfhg=dfh.groupby("Local authority name (2020)").mean()


In [8]:
dfu2 = dfu.rename(columns={'Local Authority': 'Local authority name (2020)'})

In [9]:
df = pd.merge(dfhg, dfu2, how= 'left' , on='Local authority name (2020)')

In [10]:
df.drop('Unnamed: 12', axis=1, inplace=True)

In [11]:
df = df.rename(columns={"Jan 2021 to Dec 2021":"Jan_2021_to_Dec_2021"})

In [12]:
from shapely.geometry import Point
%matplotlib inline

In [13]:
shapefile = gpd.read_file("Data_excel/LAD_MAY_2021_UK_BFE_V2.shp")

In [14]:
wq = Queen.from_dataframe(shapefile)
wq

D:\conda\envs\sds2022\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 8 disconnected components.
 There are 6 islands with ids: 43, 49, 325, 332, 334, 352.
  warnings.warn(message)


In [15]:
shapefile = shapefile.rename(columns={"LAD21NM":"Local authority name (2020)"})
shapefile.shape

(374, 10)

In [16]:
df_shape = pd.merge(df, shapefile, how= 'left' , on='Local authority name (2020)')


In [17]:
df_shape.columns

Index(['Local authority name (2020)', 'Asthma', 'Atrial fibrillation',
       'Cancer diagnosis since 2003', 'Cardiovascular disease prevention',
       'Chronic kidney disease', 'COPD', 'Coronary heart disease', 'Dementia',
       'Depression', 'Diabetes', 'Epilepsy', 'Heart failure',
       'High blood pressure (hypertension)', 'Learning disabilities',
       'Obesity', 'Osteoporosis', 'Peripheral arterial disease',
       'Rheumatoid arthritis', 'Schizophrenia, bipolar disorder & psychoses',
       'Stroke and transient ischaemic attack', 'Local Authority Code',
       'Jan 2019 to Dec 2019', 'Apr 2019 to Mar 2020', 'Jul 2019 to Jun 2020',
       'Oct 2019 to Sep 2020', 'Jan 2020 to Dec 2020', 'Apr 2020 to Mar 2021',
       'Jul 2020 to Jun 2021', 'Oct 2020 to Sep 2021', 'Jan_2021_to_Dec_2021',
       'Apr 2021 to Mar 2022', 'OBJECTID', 'LAD21CD', 'BNG_E', 'BNG_N', 'LONG',
       'LAT', 'SHAPE_Leng', 'SHAPE_Area', 'geometry'],
      dtype='object')

In [18]:
shapefile.head()

,OBJECTID,LAD21CD,Local authority name (2020),BNG_E,BNG_N,LONG,LAT,SHAPE_Leng,SHAPE_Area,geometry
0,1,E06000001,Hartlepool,447160,531474,-1.27018,54.676140,66110.011028,9.835107e+07,"POLYGON ((447213.900 537036.104, 447228.798 53..."
1,2,E06000002,Middlesbrough,451141,516887,-1.21099,54.544670,41055.789790,5.455359e+07,"POLYGON ((448489.897 522071.798, 448592.597 52..."
2,3,E06000003,Redcar and Cleveland,464361,519597,-1.00608,54.567520,105292.103526,2.537854e+08,"POLYGON ((455525.931 528406.654, 455724.632 52..."
3,4,E06000004,Stockton-on-Tees,444940,518183,-1.30664,54.556911,108085.190140,2.097308e+08,"POLYGON ((444157.002 527956.304, 444165.898 52..."
4,5,E06000005,Darlington,428029,515648,-1.56835,54.535339,107203.152334,1.974778e+08,"POLYGON ((423496.602 524724.299, 423497.204 52..."


In [19]:
smaller_df_shape = df_shape[["Local authority name (2020)", "Jan_2021_to_Dec_2021","Asthma", "Cancer diagnosis since 2003",
                             "Diabetes", "Learning disabilities", "Obesity", "Depression",
                             ]].copy()  

In [20]:
smaller_df_shape

,Local authority name (2020),Jan_2021_to_Dec_2021,Asthma,Cancer diagnosis since 2003,Diabetes,Learning disabilities,Obesity,Depression
0,Adur,3.8,7.441750,4.079375,8.720375,0.524750,11.687875,16.908875
1,Allerdale,3.5,7.407000,4.287917,8.325417,0.559917,13.555083,13.144917
2,Amber Valley,3.4,7.279687,3.798563,7.190437,0.659000,12.386188,13.793437
3,Arun,3.8,7.211211,4.620105,8.569421,0.622526,10.531105,12.221158
4,Ashfield,3.6,6.817750,3.415188,8.407750,0.720500,14.900625,13.051813
...,...,...,...,...,...,...,...,...
309,Worthing,4.4,7.481231,3.966538,7.717077,0.634077,8.815692,14.556231
310,Wychavon,3.3,7.019842,4.436579,7.419316,0.423684,13.340789,11.743684
311,Wyre,3.9,7.887214,4.465429,7.923357,0.428643,14.678357,15.446071
312,Wyre Forest,4.3,7.749571,4.481429,8.779429,0.632071,14.558429,17.790143


In [21]:
smaller_df_shape=smaller_df_shape.dropna(axis=0)

In [22]:
smaller_df_shape.shape

(312, 8)

In [23]:
smaller_df_shape = smaller_df_shape.rename(columns={
                                    "Cancer diagnosis since 2003":"Cancer_diagnosis_since_2003",
                                    "Cardiovascular disease prevention":"Cardiovascular_disease_prevention",
                                   "Learning disabilities":"Learning_disabilities",})

In [24]:
smaller_df_shape.head()

#forgot to add geometry to the smaller dataframe

,Local authority name (2020),Jan_2021_to_Dec_2021,Asthma,Cancer_diagnosis_since_2003,Diabetes,Learning_disabilities,Obesity,Depression
0,Adur,3.8,7.441750,4.079375,8.720375,0.524750,11.687875,16.908875
1,Allerdale,3.5,7.407000,4.287917,8.325417,0.559917,13.555083,13.144917
2,Amber Valley,3.4,7.279687,3.798563,7.190437,0.659000,12.386188,13.793437
3,Arun,3.8,7.211211,4.620105,8.569421,0.622526,10.531105,12.221158
4,Ashfield,3.6,6.817750,3.415188,8.407750,0.720500,14.900625,13.051813


In [25]:
#do i merge it via shapefile.merge?

In [26]:
gdf = shapefile.merge(smaller_df_shape, how="left", on="Local authority name (2020)")

#got it. Make the shapefile the 'subject' if you will

In [27]:
gdf = gdf.dropna(axis=0)

In [28]:
gdf.shape

(305, 17)

In [29]:
gdf=gdf.drop([305,306,307,308])

In [30]:
gdf.shape

(301, 17)

In [31]:
smaller_df_shape.columns

Index(['Local authority name (2020)', 'Jan_2021_to_Dec_2021', 'Asthma',
       'Cancer_diagnosis_since_2003', 'Diabetes', 'Learning_disabilities',
       'Obesity', 'Depression'],
      dtype='object')

In [32]:
gdf[~gdf.index.isin(gdf_par_2.index)]

NameError: name 'gdf_par_2' is not defined

In [ ]:
gdf_par.shape

In [ ]:
gdf_par_2.shape

In [ ]:
gdf_par.count()

In [ ]:
gdf["Local authority name (2020)"]

In [ ]:
print(gdf_par["Local authority name (2020)"])

In [ ]:
unemployment_2021 = gdf['Jan_2021_to_Dec_2021']

In [ ]:
y = np.array(unemployment_2021)

In [ ]:
y.shape = (len(unemployment_2021),1)

In [ ]:
X = []
X.append(gdf['Asthma'])
X.append(gdf['Cancer_diagnosis_since_2003'])
X.append(gdf['Depression'])
X.append(gdf['Diabetes'])
X.append(gdf['Learning_disabilities'])
X.append(gdf['Obesity'])



In [ ]:
X=np.array(X).T

In [ ]:
g_y = gdf['Jan_2021_to_Dec_2021'].values.reshape((-1,1))
g_X = gdf[["Asthma","Cancer_diagnosis_since_2003",
                 "Diabetes","Learning_disabilities",
               "Obesity", "Depression"]].values
u = gdf['LONG']
v = gdf['LAT']


g_coords = list(zip(u,v))

In [ ]:
bw = mgwr.sel_bw.Sel_BW(g_coords, 
                  g_y, # 
                  g_X, # 
                  fixed=False, # True for fixed bandwidth and false for adaptive bandwidth
                  spherical=True)

In [ ]:
bw.search()

In [ ]:
model = mgwr.gwr.GWR(g_coords, # a collection of n sets of x and y coordinates
                     g_y, # dependent variable
                     g_X, # independent variables
                     bw.bw[0]) # bandwidth value  ( we use N nearest neighbours)

In [ ]:
results = model.fit()
results.summary()

In [ ]:
print(results.params.shape)

In [ ]:
filtered_estimates_95 = results.filter_tvals(alpha=.05)
filtered_estimates_95

In [ ]:
data_params = pd.DataFrame(filtered_estimates_95)
data_localR2 = pd.DataFrame(results.localR2)

In [ ]:
gdf_par = gdf_par.dropna(axis=0)

In [ ]:
gdf_map=pd.DataFrame(gdf["Local authority name (2020)"])
gdf_par = gdf_map.assign(intercept=data_params[0],
                 Asthma=data_params[1],
                 Cancer_diagnosis_since_2003=data_params[2],
                        Diabetes=data_params[3],
                        Learning_disabilities=data_params[4],
                      Obesity=data_params[5],
                               Depression=data_params[6],
                        localR2=data_localR2[0])

In [ ]:
gdf_par_2 = gdf_map.assign(intercept=data_params[0],
                 Asthma=data_params[1],
                 Cancer_diagnosis_since_2003=data_params[2],
                        Diabetes=data_params[3],
                        Learning_disabilities=data_params[4],
                      Obesity=data_params[5],
                               Depression=data_params[6],
                        localR2=data_localR2[0])

In [ ]:
gdf_par_2  = shapefile.merge(gdf_par_2, how="left", on="Local authority name (2020)")

In [ ]:
gdf_par  = shapefile.merge(gdf_par, how="left", on="Local authority name (2020)")

In [ ]:
gdf_par.shape

In [ ]:
gdf_par_2 = gdf_par_2.dropna(axis=0)

In [ ]:
gdf_par_2.shape

In [ ]:
gdf_par.shape

In [ ]:
gdf_map.shape

In [ ]:
shapefile.columns

In [ ]:
england_shp = shapefile.merge(gdf_par,left_on="Local authority name (2020)", right_on= "Local authority name (2020)")

In [ ]:
england_shp.shape

In [ ]:
gdf_par_merge = shapefile.merge(gdf_par, how="left", on="Local authority name (2020)")

In [ ]:
gdf_par_merge = gdf_par_merge.dropna(axis=0)

In [ ]:
gdf_par_merge.shape

In [ ]:
gdf_non_sig = england_shp.assign(toplot=filtered_estimates_95.T[i])

In [ ]:
# First, let's create empty set of subplots with 2 rows and 3 columns
f,ax=plt.subplots(4,2,figsize=(20,10), subplot_kw=dict(aspect='equal'))
# Flatten them
ax = ax.flatten()
# Define the title of our plots
titles = ['Intercept %',"Asthma %","Cancer Diagnosis since 2003 %","Diabetes %",
        "Learning Disabilities %","Obesity %", "Depression %"]

# Here, we are creating loop for our parameter estimates 
for i,row in enumerate(filtered_estimates_95.T):
    # We want to plot all the non-significant estimates in grey
    gdf_non_sig = gdf_par_merge.assign(toplot=filtered_estimates_95.T[i])
    (gdf_non_sig.query('toplot==0')
     .sort_values('toplot')
     .plot(color='grey',ax=ax[i],alpha=.2))
    # and assign colours only to the significant estimates
    (gdf_non_sig.query('toplot!=0')
     .sort_values('toplot')
     .plot('toplot',
           cmap='Blues',
           ax=ax[i],
           legend=True))
    
    ax[i].set_title(titles[i], fontsize=10)
    
    ax[i].set_xticklabels([])
    ax[i].set_yticklabels([])
    ax[i].set_xticks([])
    ax[i].set_yticks([])
 
#For the last one, local R2, we can just map all of them regarding the significance

(gdf_par_merge.assign(r2=results.localR2)
 .sort_values('r2')
 .plot('r2',
       ax=ax[-1],
       legend=True,
       
       cmap='Blues'))
    
ax[-1].set_xticklabels([])
ax[-1].set_yticklabels([])
ax[-1].set_xticks([])
ax[-1].set_yticks([])
    
ax[-1].set_title('Local R2', fontsize=10
                )

    

In [ ]:
gdf_par_2[gdf.isnull().any(axis=1)]

In [ ]:
gwr_selector = Sel_BW(g_coords, g_y, g_X)
gwr_bw = gwr_selector.search(bw_min=2)
print(gwr_bw)
gwr_results = GWR(g_coords, g_y, g_X, gwr_bw).fit()


In [ ]:
mgwr_selector = Sel_BW(g_coords, g_y, g_X, multi=True)
mgwr_bw = mgwr_selector.search(multi_bw_min=[2])
print(mgwr_bw)
mgwr_results = MGWR(g_coords, g_y, g_X, mgwr_selector).fit()

In [ ]:
# show gwr results here
gwr_results.summary()

In [ ]:
mgwr_results.summary()

In [ ]:
gdf['gwr_intercept'] = gwr_results.params[:,0]
gdf['gwr_Asthma'] = gwr_results.params[:,1]
gdf['gwr_Cancer_diagnosis_since_2003'] = gwr_results.params[:,2]
gdf['gwr_Diabetes'] = gwr_results.params[:,3]
gdf['gwr_Learning_Disabilities'] = gwr_results.params[:,4]
gdf['gwr_Obesity'] = gwr_results.params[:,5]
gdf['gwr_Depression'] = gwr_results.params[:,6]

In [ ]:
df2_map_shape['mgwr_intercept'] = mgwr_results.params[:,0]
df2_map_shape['mgwr_med_income'] = mgwr_results.params[:,1]
df2_map_shape['gwr_med_rent'] = gwr_results.params[:,2]
df2_map_shape['mgwr_totalSS_arrest'] = mgwr_results.params[:,3]


In [ ]:
Local authority name (2020)', 'Jan_2021_to_Dec_2021', 'Asthma',
       'Cancer_diagnosis_since_2003', 'Diabetes', 'Learning_disabilities',
       'Obesity', 'Depression'],
      dtype='object